In [1]:
# Create or update conda environment
# !conda env create -f ../environment.yaml
# !conda env update -f ../environment.yaml

In [2]:
# Activate conda environment
!conda activate amadeus-ex-machina

In [1]:
# System imports
import sys
import os

# Add the parent directory of 'notebooks' to sys.path
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))  # Move one level up
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

# Test if the path is correctly added
print(f"Parent directory added to sys.path: {parent_dir}")

# Import packages
import json
import torchaudio
import torch
import torch.optim as optim
import torch.nn as nn
from torchaudio.transforms import MelSpectrogram, AmplitudeToDB
from torch.utils.data import Dataset, DataLoader

# Import models
from models.CRNN import CRNNModel
from models.CNN import CNNModel
from models.RNN import RNNModel
from models.AudioDataset import ChordDataset

json_file = "../datagen/chords/chord_ref.json"
audio_dir = "../datagen/chords/midi/"

Parent directory added to sys.path: /Users/dananabulsi/Desktop/CODING/GitHub-Repos/amadeus-ex-machina


ImportError: dlopen(/opt/anaconda3/envs/amadeus-ex-machina/lib/python3.11/lib-dynload/cmath.cpython-311-darwin.so, 0x0002): symbol not found in flat namespace '__PyModule_Add'

In [4]:
# Data preprocessing

# Load the JSON metadata
with open(json_file, "r") as f:
    metadata = json.load(f)

# Create a mapping for chord classes to integers
chord_classes = sorted({value["chord_class"] for value in metadata.values()})
chord_class_to_idx = {chord: idx for idx, chord in enumerate(chord_classes)}
print("Chord Class to Index Mapping:", chord_class_to_idx)

# Create Dataset and DataLoader
dataset = ChordDataset(metadata, audio_dir, chord_class_to_idx)

# Split into training and validation datasets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Move model to the selected device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Get the number of classes
num_classes = len(chord_class_to_idx)

Chord Class to Index Mapping: {'dim': 0, 'maj7': 1, 'sus2': 2}


In [5]:
# Modular training and validation function

def training_validation(model):
    # Loss function
    criterion = nn.CrossEntropyLoss()

    # Optimizer
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Training Loop
    num_epochs = 20
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0

        # Loop over the batches in the training dataset
        for spectrograms, labels in train_loader:
            # Move data to the selected device
            spectrograms, labels = spectrograms.to(device), labels.to(device)

            # Zero gradients from the previous step
            optimizer.zero_grad()

            # Forward pass
            outputs = model(spectrograms)

            # Compute loss
            loss = criterion(outputs, labels)

            # Backward pass
            loss.backward()

            # Update model weights
            optimizer.step()

            total_loss += loss.item()

        # Print training loss for the current epoch
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(train_loader):.4f}")

        # Evaluate on validation data after every epoch
        model.eval()  # Set model to evaluation mode
        correct = 0
        total = 0
        with torch.no_grad():  # No need to compute gradients during evaluation
            for spectrograms, labels in val_loader:
                spectrograms, labels = spectrograms.to(device), labels.to(device)
                outputs = model(spectrograms)
                _, predicted = torch.max(outputs, 1)  # Get predicted class

                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_accuracy = 100 * correct / total
        print(f"Validation Accuracy: {val_accuracy:.2f}%")

In [6]:
# Initialize CRNN model
crnn_model = CRNNModel(input_channels=2, num_classes=num_classes, hidden_size=128).to(device)
training_validation(crnn_model)

Epoch [1/20], Loss: 1.0522
Validation Accuracy: 0.00%
Epoch [2/20], Loss: 0.7225
Validation Accuracy: 0.00%
Epoch [3/20], Loss: 0.6021
Validation Accuracy: 0.00%
Epoch [4/20], Loss: 0.5103
Validation Accuracy: 0.00%
Epoch [5/20], Loss: 0.4194
Validation Accuracy: 0.00%
Epoch [6/20], Loss: 0.2809
Validation Accuracy: 0.00%
Epoch [7/20], Loss: 0.1616
Validation Accuracy: 0.00%
Epoch [8/20], Loss: 0.1042
Validation Accuracy: 0.00%
Epoch [9/20], Loss: 0.0653
Validation Accuracy: 0.00%
Epoch [10/20], Loss: 0.0400
Validation Accuracy: 0.00%
Epoch [11/20], Loss: 0.0294
Validation Accuracy: 0.00%
Epoch [12/20], Loss: 0.0256
Validation Accuracy: 0.00%
Epoch [13/20], Loss: 0.0171
Validation Accuracy: 0.00%
Epoch [14/20], Loss: 0.0107
Validation Accuracy: 0.00%
Epoch [15/20], Loss: 0.0081
Validation Accuracy: 0.00%
Epoch [16/20], Loss: 0.0064
Validation Accuracy: 0.00%
Epoch [17/20], Loss: 0.0051
Validation Accuracy: 0.00%
Epoch [18/20], Loss: 0.0041
Validation Accuracy: 0.00%
Epoch [19/20], Loss

In [7]:
# Initialize CNN model
cnn_model = CNNModel(input_channels=2, num_classes=num_classes).to(device)
training_validation(cnn_model)

Epoch [1/20], Loss: 1.0743
Validation Accuracy: 0.00%
Epoch [2/20], Loss: 0.9833
Validation Accuracy: 0.00%
Epoch [3/20], Loss: 0.9092
Validation Accuracy: 0.00%
Epoch [4/20], Loss: 0.8458
Validation Accuracy: 0.00%
Epoch [5/20], Loss: 0.7982
Validation Accuracy: 0.00%
Epoch [6/20], Loss: 0.7609
Validation Accuracy: 0.00%
Epoch [7/20], Loss: 0.7320
Validation Accuracy: 0.00%
Epoch [8/20], Loss: 0.7081
Validation Accuracy: 0.00%
Epoch [9/20], Loss: 0.6881
Validation Accuracy: 0.00%
Epoch [10/20], Loss: 0.6718
Validation Accuracy: 0.00%
Epoch [11/20], Loss: 0.6566
Validation Accuracy: 0.00%
Epoch [12/20], Loss: 0.6426
Validation Accuracy: 0.00%
Epoch [13/20], Loss: 0.6313
Validation Accuracy: 0.00%
Epoch [14/20], Loss: 0.6193
Validation Accuracy: 0.00%
Epoch [15/20], Loss: 0.6078
Validation Accuracy: 0.00%
Epoch [16/20], Loss: 0.5967
Validation Accuracy: 0.00%
Epoch [17/20], Loss: 0.5864
Validation Accuracy: 0.00%
Epoch [18/20], Loss: 0.5760
Validation Accuracy: 0.00%
Epoch [19/20], Loss

In [8]:
# Calculate the flattened size of the spectrograms
for spectrograms, _ in train_loader:
    rnn_input_size = spectrograms.view(spectrograms.size(0), -1).size(1)
    break

# Initialize the RNN model
rnn_model = RNNModel(input_size=rnn_input_size, hidden_size=128, output_size=num_classes).to(device)
training_validation(rnn_model)

Epoch [1/20], Loss: 1.3357
Validation Accuracy: 0.00%
Epoch [2/20], Loss: 0.1893
Validation Accuracy: 0.00%
Epoch [3/20], Loss: 0.0373
Validation Accuracy: 0.00%
Epoch [4/20], Loss: 0.0818
Validation Accuracy: 0.00%
Epoch [5/20], Loss: 0.0679
Validation Accuracy: 0.00%
Epoch [6/20], Loss: 0.0008
Validation Accuracy: 0.00%
Epoch [7/20], Loss: 0.0000
Validation Accuracy: 0.00%
Epoch [8/20], Loss: 0.0001
Validation Accuracy: 0.00%
Epoch [9/20], Loss: 0.0002
Validation Accuracy: 0.00%
Epoch [10/20], Loss: 0.0004
Validation Accuracy: 0.00%
Epoch [11/20], Loss: 0.0007
Validation Accuracy: 0.00%
Epoch [12/20], Loss: 0.0011
Validation Accuracy: 0.00%
Epoch [13/20], Loss: 0.0014
Validation Accuracy: 0.00%
Epoch [14/20], Loss: 0.0015
Validation Accuracy: 0.00%
Epoch [15/20], Loss: 0.0012
Validation Accuracy: 0.00%
Epoch [16/20], Loss: 0.0009
Validation Accuracy: 0.00%
Epoch [17/20], Loss: 0.0006
Validation Accuracy: 0.00%
Epoch [18/20], Loss: 0.0004
Validation Accuracy: 0.00%
Epoch [19/20], Loss